In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [10]:
final_feats = pd.DataFrame([1, 2, 3])
top = pd.DataFrame([4, 5, 6])
final_feats = pd.concat([final_feats, top], axis=1)
#final_feats
len(final_feats)

3

In [51]:
### считываем результаты наших классификаторов, соединяем в один датафрэйм
train_files_list = ['rf_13_subset_stacking_class_mean_train.csv',
                   'rf_21_subset_stacking_class_mean_train.csv',
                   'rf_19allfeats_stacking_class_mean_train.csv',
                   'rf_float_stacking_class_mean_train.csv',
                   'gbclf_30feats_stacking_class_mean_train.csv',
                   'xgbclf_30feats_stacking_class_mean_train.csv',
                   'gbclf_19feats_stacking_class_mean_train.csv',
                   'xgbclf_19feats_stacking_class_mean_train.csv',
                   'rf_30feats_stacking_class_mean_train.csv']
X_train = pd.DataFrame()
for file in train_files_list:
    cur = pd.read_csv(file, header=-1)
    X_train = pd.concat([X_train, cur], axis=1)

test_files_list = ['rf_13_subset_stacking_class_mean_test.csv',
                  'rf_21_subset_stacking_class_mean_test.csv',
                  'rf_19allfeats_stacking_class_mean_test.csv',
                  'rf_float_stacking_class_mean_test.csv',
                  'gbclf_30feats_stacking_class_mean_test.csv',
                  'xgbclf_30feats_stacking_class_mean_test.csv',
                  'gbclf_19feats_stacking_class_mean_test.csv',
                  'xgbclf_19feats_stacking_class_mean_test.csv',
                  'rf_30feats_stacking_class_mean_test.csv'
                  ]
X_test = pd.DataFrame()
for file in test_files_list:
    cur = pd.read_csv(file, header=-1)
    X_test = pd.concat([X_test, cur], axis=1)

y_train = pd.read_csv('y_train.csv', sep=';', names = ['class'])
y_train = y_train.squeeze()

In [44]:
skf_cv = StratifiedKFold(n_splits=5)

##### лэйблим входные данные

In [52]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [53]:
X_train_ = enc.fit_transform(X_train)

In [54]:
X_train_ = pd.DataFrame(X_train_.toarray())

In [55]:
X_train_.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [56]:
X_test_ = pd.DataFrame(enc.transform(X_test).toarray())
X_test_.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


##### первый вариант метаклассификатора - LogReg

In [57]:
log_reg = LogisticRegression()
cross_val_score(log_reg, X_train_, y_train, scoring='accuracy', cv = skf_cv).mean()

0.64661207369243034

In [37]:
###попробуем подтюнить логистическую регрессию
### функции для оптимального поиска гиперпараметров
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def score(params):
    lr = LogisticRegression(**params)
    #skf_cv = StratifiedKFold(n_splits=5)
    score = -cross_val_score(lr, X_train_, y_train, scoring='accuracy', cv=skf_cv).mean()
    print ("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}

def optimize(trials):
    space = {
            'C' : hp.choice('C', [0.1, 1, 10, 25, 100]),
            'solver' : hp.choice('solver', ['lbfgs', 'newton-cg', 'sag']),
            'multi_class': hp.choice('multi_class', ['multinomial', 'ovr'])
    }
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=50)

    print(best)
    


In [38]:
### пробуем дотюнивать на отобранных признаках
trials = Trials()
optimize(trials)

	Score -0.6428883256937574




C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_

	Score -0.6434618026909579




C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_

	Score -0.6443226418962784




...

	Score -0.6428883256937574


	Score -0.6426054937956203


	Score -0.6443238810664242


	Score -0.6443238810664242


	Score -0.6443238810664242


	Score -0.6443238810664242


	Score -0.6443238810664242


	Score -0.6426054937956203


	Score -0.6443238810664242


	Score -0.6443238810664242


	Score -0.6443238810664242


	Score -0.6426054937956203


	Score -0.6443238810664242


	Score -0.6443238810664242


	Score -0.6443238810664242


	Score -0.6414536078065639


	Score -0.6426054937956203


	Score -0.6443238810664242


	Score -0.6414536078065639


	Score -0.6443238810664242


	Score -0.6423140288496755


	Score -0.6443238810664242


	Score -0.6434618026909579


	Score -0.6426054937956203


	Score -0.6440356966690609


	Score -0.6443238810664242




C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_

	Score -0.6426013851715145


	Score -0.6431752697396684


	Score -0.6443238810664242




C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
C:\Users\Lampard\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_

	Score -0.6434618026909579


	Score -0.6423144411256035


{'solver': 1, 'C': 1, 'multi_class': 0}


In [62]:
log_reg.fit(X_train_, y_train)
ans1 = log_reg.predict(X_test_)
np.savetxt('log_reg_ans_final.csv', ans1, fmt='%i', delimiter='\n')

##### второй вариант метаклассификатора - голосование

In [58]:
x_ts = X_test.values
x_tr = X_train.values

In [59]:
ans2 = np.array([0 for i in range(len(x_ts))])
for i in range(len(x_ts)):
    ans2[i] = np.bincount(x_ts[i]).argmax()
    #print(ans2[i])

In [60]:
ans3 = np.array([0 for i in range(len(x_tr))])
for i in range(len(x_tr)):
    ans3[i] = np.bincount(x_tr[i]).argmax()
    #print(ans2[i])

In [49]:
np.savetxt('voting_ans.csv', ans1, fmt='%i', delimiter='\n')

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(ans3, y_train.as_matrix())

0.64087130983089713

##### третий вариант метаклассификатора - SVM

In [50]:
from sklearn.svm import SVC
svc = SVC()
cross_val_score(svc, X_train_, y_train, scoring='accuracy', cv = skf_cv).mean()

0.64289487617941377